In [30]:
import sagemaker

bucket = 'plansoft-sagemaker-mask-detect'
prefix = ''
data_directory = 'data/training'
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::960704241809:role/plansoft-research-sagemaker


In [31]:
inputs = 's3://' + bucket + '/' + prefix + data_directory
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://plansoft-sagemaker-mask-detect/data/training


In [32]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                role=role,
                framework_version='1.6.0',
                train_instance_count=1,
                train_instance_type='ml.g4dn.2xlarge',
                # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                #                            bptt, dropout, tied, seed, log_interval
                hyperparameters={
                    'epochs': 10
                })

In [ ]:
estimator.fit({'training': inputs})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-16 10:57:38 Starting - Starting the training job...
2020-10-16 10:57:40 Starting - Launching requested ML instances.....

In [17]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [18]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.6.0',
                     entry_point='generate.py',
                     predictor_cls=JSONPredictor)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [19]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.2xlarge')

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------------!

In [20]:
input = {
    'url': 'https://i.insider.com/5e71267bc4854010b62f5925?width=1100&format=jpeg&auto=webp'
}
response = predictor.predict(input)
print(response)

[{'boxes': [[595.0823364257812, 317.06536865234375, 761.9072265625, 538.1050415039062], [423.21466064453125, 289.1833190917969, 540.6068115234375, 433.2112121582031], [825.971435546875, 491.39019775390625, 948.559814453125, 625.0926513671875], [184.1943359375, 349.5162048339844, 294.6078186035156, 514.2473754882812], [145.6531219482422, 397.62152099609375, 173.2164764404297, 428.63775634765625], [220.3684539794922, 358.98419189453125, 250.04345703125, 393.36395263671875], [141.3076934814453, 398.3751220703125, 182.16542053222656, 451.91064453125], [630.07666015625, 719.7356567382812, 650.1061401367188, 737.9102783203125], [412.44512939453125, 251.78726196289062, 557.0693359375, 500.2672119140625], [253.27320861816406, 458.44921875, 443.42889404296875, 774.9124145507812], [811.2346801757812, 428.7839050292969, 956.8544311523438, 669.8208618164062], [145.18846130371094, 408.3499450683594, 180.65330505371094, 434.8433837890625], [509.4392395019531, 199.46583557128906, 883.59765625, 825.0]

In [22]:
sagemaker_session.delete_endpoint(predictor.endpoint)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:eu-west-1:960704241809:endpoint/pytorch-inference-2020-10-15-15-29-45-123".